In [1]:
import numpy as np
import pandas as pd
import os 
import time
import deepdish.io as dd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

from svi import SVI
import models as m
import datasets as d

# Load and Prepare Data

In [2]:
data = dd.load("sequenced_data_for_VAE_lenght-160_stride-10.h5")

In [3]:
X,y = data["X"], data["y"]

In [4]:
X.shape

torch.Size([83680, 160])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=666, shuffle=True)

In [6]:
X_train.shape

(66944, 160)

In [7]:
y_train.shape

(66944,)

In [8]:
def RNNPreporces_batch(X, y, batch_size):
    length = (X.shape[0]//batch_size)*batch_size
    return X[:length, :], y[:length]

In [9]:
X_train, y_train = RNNPreporces_batch(X_train, y_train, batch_size=200)
X_test, y_test = RNNPreporces_batch(X_test, y_test, batch_size=200)

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(66800, 160)
(66800,)
(16600, 160)
(16600,)


In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
train = d.H_alphaSequences(X_train, y_train)
test = d.H_alphaSequences(X_test, y_test)

In [13]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=200, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test, batch_size=200, shuffle=True)

# Model setup

In [14]:
torch.cuda.is_available()

True

In [15]:
torch.cuda.get_device_name()

'GeForce GTX 1070'

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [17]:
model_name = "DeepLSTM_VAE_NLL_100ep_2hidden-64_lr-1e-3_bs-200"

In [18]:
#deepmodel.eval()

In [19]:
deepmodel = m.DeepLSTM_VAE(
                        sequence_len=160, 
                        n_features=1,
                        latent_dim=15,
                        hidden_size=64,
                        num_layers=2,
                        batch_size=200,
                        use_cuda=True
                        )
svi = SVI(
        vae_model=deepmodel,
        optimizer=torch.optim.Adam(deepmodel.parameters(), lr=1e-3),
        loss_function="GaussianNLL",
        verbose=True
        )

cuda:0


In [ ]:
loss_hist = svi(epochs=30, train_loader=train_loader, validation_loader=test_loader, flatten=False)

Epoch [1/30], average_loss:227.7323, validation_loss:227.0238
Epoch [2/30], average_loss:230.2919, validation_loss:228.5594


In [ ]:
plt.figure(figsize=(15,8))
plt.title("train and validation loss")
epoch_axis = np.arange(len(svi.loss_history["train"]))
plt.plot(epoch_axis, np.array(svi.loss_history["train"]), lw=0.5) #np.arange(len(svi.loss_history["validation"]))*10
plt.plot(np.arange(len(svi.loss_history["validation"]))*10, svi.loss_history["validation"])
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
torch.save(svi.model, f"VAE_saves_and_logs/{model_name}.pt")
torch.save(svi.optimizer, f"VAE_saves_and_logs/{model_name}_optimizer.pt")
dd.save(f"VAE_saves_and_logs/{model_name}_loss_history.h5", svi.loss_history)

# Evaluation

In [ ]:
print(device)
X,_ = next(iter(test_loader))
output, mu, sigma = svi.model(X.to(device))
#X_hat = output.cpu().detach().numpy()
mu_hat = output[0].cpu().detach().numpy()
sigma = output[1].cpu().detach().numpy()

fig, axes = plt.subplots(10, 2, figsize=(15,50))
fig.tight_layout()
for i in range(10):
    axes[i][0].plot(X[i])
    axes[i][0].set_title('Original image')
    #axes[i][1].plot(X_hat[i])
    axes[i][1].plot(mu_hat[i])
    axes[i][1].fill_between(np.arange(160), mu_hat[i] + 2 * sigma[i], mu_hat[i] - 2 * sigma[i], alpha = 0.5, label="Uncertainty")
    axes[i][1].set_title('Reconstructed image')

# Load Model

In [ ]:
mod = torch.load(f"VAE_saves_and_logs/{model_name}.pt")

In [ ]:
mod.eval()

In [ ]:
z, mu, sigma = mod.encoder(X.to(device))

In [ ]:
X_1, cl = test.X, test.y
Z_hat, mu, sigma = mod.encoder(X_1.to(device)) #.view(-1,28*28)
Z_hat = Z_hat.cpu().detach().numpy()
mu = mu.cpu().detach().numpy()

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
#ax.scatter(Z_hat[:,0], Z_hat[:,1], Z_hat[:,2], c=cl, lw=3)
sc = ax.scatter(mu[:,0], mu[:,1], mu[:,2], c=cl, lw=3)

In [ ]:
# když přidám colorbar k obrázku tak se nehezky deformuje, tak je tady zvlášť
plt.colorbar(sc, shrink=1)

In [ ]:
test.y